In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from glob import glob

import numpy as np
import pandas as pd
idx = pd.IndexSlice
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

import copy

import geopandas as gpd

from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

import gpflow
import tensorflow as tf
import sys
code_dir = '/cluster/home/kheuto01/code/zero-inflated-gp/'
sys.path.append(code_dir)
from math import radians, cos, sin, asin, sqrt
from onoffgpf import OnOffSVGP, OnOffLikelihood

import pickle

from math import radians, cos, sin, asin, sqrt

import copy
import sys

import geopandas as gpd
import numpy as np

import gpflow


code_dir = '/cluster/home/kheuto01/code/zero-inflated-gp/'
sys.path.append(code_dir)

code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/'
sys.path.append(code_dir)

from onoffgpf import OnOffSVGP, OnOffSVGPPoiMC, OnOffLikelihood
gpflow.config.default_float()


from zinf_gp.metrics import normcdf, fixed_top_X

2023-03-02 10:19:36.619530: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 10:19:36.775878: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-02 10:19:36.781591: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-02 10:19:36.781606: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:

data_dir='/cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/'

log_dir='/cluster/tufts/hugheslab/kheuto01/opioid/logs/converge/'

run_template = '{time}_{loc}_{model}_{start_year}_{cov}_{num_inducing}_{lr}'

In [3]:
times = ['annual', 'quarter','semi']
locs = ['tract','town','group']
start_years = [ 2000,2010]
covs = [ '-auto','-auto-svi','all']
models = ['normal', 'poisson']
learning_rates = [0.1,0.01,0.001]
inducing_points = [400,200]

In [7]:
this_run = run_template.format(time='quarter',loc='tract',model='normal',start_year='2010',cov='all',num_inducing=200,lr=0.01)
pd.read_csv(os.path.join(log_dir,this_run,'stats.csv'))

FileNotFoundError: [Errno 2] No such file or directory: '/cluster/tufts/hugheslab/kheuto01/opioid/logs/larger_zults/quarter_tract_normal_2010_all_200_0.01/stats.csv'

In [6]:
time = 'annual'
loc='tract'
cov='all'
for num_inducing in inducing_points:
    for model in models:
        for lr in learning_rates:
        

            for start_year in start_years:    
                best_bpr_2019, best_bpr_2020 = -100, -100

                for lr in learning_rates:
                    this_run = run_template.format(time=time,loc=loc,
                                                   model=model,start_year=start_year,
                                                   cov=cov,
                                                   num_inducing=num_inducing,lr=lr)
                    try:
                        stats = pd.read_csv(os.path.join(log_dir,this_run,'stats.csv'))
                    except(FileNotFoundError):
                        print(f'Broke {this_run}')
                        continue
                    last_row = stats.iloc[-1,:]

                    if last_row['bpr_100_0'] > best_bpr_2019:
                        best_bpr_2019 = copy.copy(last_row['bpr_100_0'])
                    if last_row['bpr_100_1'] > best_bpr_2020:
                        best_bpr_2020 = copy.copy(last_row['bpr_100_1'])
                print(f'{this_run} 2019:{best_bpr_2019}, 2020:{best_bpr_2020}')
                
            

annual_tract_normal_2000_all_400_0.001 2019:0.5149572649572649, 2020:0.5270541082164328
annual_tract_normal_2010_all_400_0.001 2019:0.5021367521367521, 2020:0.5450901803607212
annual_tract_normal_2000_all_400_0.001 2019:0.5149572649572649, 2020:0.5270541082164328
annual_tract_normal_2010_all_400_0.001 2019:0.5021367521367521, 2020:0.5450901803607212
annual_tract_normal_2000_all_400_0.001 2019:0.5149572649572649, 2020:0.5270541082164328
annual_tract_normal_2010_all_400_0.001 2019:0.5021367521367521, 2020:0.5450901803607212
annual_tract_poisson_2000_all_400_0.001 2019:0.5, 2020:0.5230460921843686
annual_tract_poisson_2010_all_400_0.001 2019:0.5192307692307693, 2020:0.5310621242484971
annual_tract_poisson_2000_all_400_0.001 2019:0.5, 2020:0.5230460921843686
annual_tract_poisson_2010_all_400_0.001 2019:0.5192307692307693, 2020:0.5310621242484971
annual_tract_poisson_2000_all_400_0.001 2019:0.5, 2020:0.5230460921843686
annual_tract_poisson_2010_all_400_0.001 2019:0.5192307692307693, 2020:0.

In [68]:
basic_results = []
for num_inducing in inducing_points:
    for model in models:
        for cov in covs:
            for start_year in start_years:
                for loc in locs:
                    for time in times:
                        
                        best_bpr_2019, best_bpr_2020 = -100, -100
                        
                        for lr in learning_rates:
                            this_run = run_template.format(time=time,loc=loc,
                                                           model=model,start_year=start_year,
                                                           cov=cov,
                                                           num_inducing=num_inducing,lr=lr)
                            try:
                                stats = pd.read_csv(os.path.join(log_dir,this_run,'stats.csv'))
                            except(FileNotFoundError):
                                print(f'Broke {this_run}')
                                data = [time, loc, start_year, cov, model, num_inducing, np.NaN, np.NaN]
                                basic_results.append(data)
                                continue
                            last_row = stats.iloc[-1,:]

                            if last_row['bpr_100_0'] > best_bpr_2019:
                                best_bpr_2019 = copy.copy(last_row['bpr_100_0'])
                            if last_row['bpr_100_1'] > best_bpr_2020:
                                best_bpr_2020 = copy.copy(last_row['bpr_100_1'])
                                
                        data = [time, loc, start_year, cov, model, num_inducing, best_bpr_2019, best_bpr_2020]
                        basic_results.append(data)
pd.DataFrame(basic_results).to_csv('~/result_sheet.csv')

In [13]:
recalc_results = []
test_years = 2
geography_col='geoid'
timestep_col='timestep'
outcome_col='deaths'
for loc in [ 'tract', 'group','town']:
    timesteps_per_year = 4

        
    file_name = f'clean_quarter_{loc}'
    data_path = os.path.join(data_dir, file_name)
    
    x_idx_cols = [geography_col, 'lat','lon', timestep_col,
          'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
          'svi_pctile',
          'neighbor_t', 'self_t-1']
    y_idx_cols = [geography_col, timestep_col, outcome_col]
    features_only = ['lat','lon', timestep_col,
                     'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
                     'svi_pctile',
                     'neighbor_t', 'self_t-1']

    data_gdf = gpd.read_file(data_path)

    last_train_year = 2018


    test_x = data_gdf[(data_gdf['year'] > last_train_year) &
                      (data_gdf['year'] <= last_train_year+test_years)][x_idx_cols]
    test_y = data_gdf[(data_gdf['year'] > last_train_year) &
                      (data_gdf['year'] <= last_train_year+test_years)][y_idx_cols]

    import pdb;pdb.set_trace()
    starting_timestep = int(test_x[timestep_col].min())

    for time in ['semi', 'annual']:
        

        
        
        for start_year in start_years:
            print('HNY')
            
            for num_inducing in inducing_points:
                for model in models:
                    for cov in covs:
                    
                        best_bpr_2019, best_bpr_2020 = -100, -100
                        
                        for lr in learning_rates:
                            this_run = run_template.format(time=time,loc=loc,
                                                           model=model,start_year=start_year,
                                                           cov=cov,
                                                           num_inducing=num_inducing,lr=lr)
                            try:
                                with open(os.path.join(log_dir,this_run,'model.mod'),'rb') as f:
                                        predictor = pickle.load(f)
                                    
                            except(FileNotFoundError):
                                print(f'Broke {this_run}')
                                data = [time, loc, start_year, cov, model, num_inducing, np.NaN, np.NaN]
                                result.append(data)
                                continue


                            xtops = []
                            for year in range(test_years):
                                xtop_year = []
                                for timestep in range(starting_timestep+year*timesteps_per_year, starting_timestep+year*timesteps_per_year+timesteps_per_year):
                                    test_x_time = test_x[test_x[timestep_col] == timestep]
                                    test_y_time = test_y[test_y[timestep_col] == timestep]
                                    _, _, _, fmean, fvar, gmean, gvar, _, _ = predictor.build_predict(test_x_time.loc[:, features_only].values)
                                    g_cond = tf.math.softplus(fmean * normcdf(gmean) + 2).numpy()
                                    pred_df = pd.Series(g_cond.squeeze(), index=test_y_time[geography_col])

                                    xtop_year.append(fixed_top_X(test_y_time.set_index(geography_col)[outcome_col], pred_df, 100))
                                xtops.append(xtop_year)
                            curr_results = {} 
                            for y, xtop in enumerate( xtops):
                                curr_results[f'bpr_100_{y}'] = copy.deepcopy(np.mean([thing[3] for thing in xtop]))
                                
                            if curr_results[f'bpr_100_0'] > best_bpr_2019:
                                best_bpr_2019 = copy.copy(curr_results[f'bpr_100_0'])
                            if curr_results[f'bpr_100_1'] > best_bpr_2020:
                                best_bpr_2020 = copy.copy(curr_results[f'bpr_100_1'])
                                

                        data = [time, loc, start_year, cov, model, num_inducing, best_bpr_2019, best_bpr_2020]
                        recalc_results.append(data)
    
        pd.DataFrame(recalc_results).to_csv(f'~/recalc_results_sheet_{loc}_{time}.csv')
pd.DataFrame(recalc_results).to_csv('~/recalc_results_sheet.csv')

> /tmp/ipykernel_325941/2076851216.py(34)<cell line: 6>()
     32 
     33     import pdb;pdb.set_trace()
---> 34     starting_timestep = int(test_x[timestep_col].min())
     35 
     36     for time in ['semi', 'annual']:

ipdb> x_timesteps
[19.0, 20.0, 19.0, 20.0, 19.0, 20.0, 19.0, 20.0]
--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
HNY


2023-02-16 18:52:05.939110: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-02-16 18:52:05.939153: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: p1cmp078.pax.tufts.edu
2023-02-16 18:52:05.939161: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: p1cmp078.pax.tufts.edu
2023-02-16 18:52:05.939306: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.47.3
2023-02-16 18:52:05.939340: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.47.3
2023-02-16 18:52:05.939345: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.47.3
2023-02-16 18:52:05.939717: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the fo

In [42]:
os.path.join(log_dir,this_run)

'/cluster/tufts/hugheslab/kheuto01/opioid/logs/new_big_run/annual_tract_poisson_2010_all_400_0.001'

In [43]:
!ls -larth /cluster/tufts/hugheslab/kheuto01/opioid/logs/new_big_run/annual_tract_poisson_2010_all_400_0.001


total 2.6M
drwxrws--- 2 kheuto01 hugheslab 4.0K Feb 14 18:00 ..
drwxrws--- 2 kheuto01 hugheslab 4.0K Feb 14 19:07 .
-rw-rw---- 1 kheuto01 hugheslab 2.2K Feb 14 19:14 stats.csv
-rw-rw---- 1 kheuto01 hugheslab 2.6M Feb 14 19:14 model.mod


In [50]:
test_y

,geoid,timestep,deaths
76,25001010100,76.0,0.0
77,25001010100,77.0,0.0
78,25001010100,78.0,1.0
79,25001010100,79.0,0.0
80,25001010100,80.0,0.0
...,...,...,...
142551,25027761402,79.0,0.0
142552,25027761402,80.0,0.0
142553,25027761402,81.0,0.0
142554,25027761402,82.0,0.0


In [69]:
starting_timestep = int(test_x[timestep_col].min())


xtops = []
for year in range(test_years):
    xtop_year = []
    for timestep in range(starting_timestep+year*timesteps_per_year, starting_timestep+year*timesteps_per_year+timesteps_per_year):
        test_x_time = test_x[test_x[timestep_col] == timestep]
        test_y_time = test_y[test_y[timestep_col] == timestep]
        _, _, _, fmean, fvar, gmean, gvar, _, _ = predictor.build_predict(test_x_time.loc[:, features_only].values)
        g_cond = tf.math.softplus(fmean * normcdf(gmean) + 2).numpy()
        pred_df = pd.Series(g_cond.squeeze(), index=test_y_time[geography_col])

        xtop_year.append(fixed_top_X(test_y_time.set_index(geography_col)[outcome_col], pred_df, 100))
    xtops.append(xtop_year)
curr_results = {} 
for y, xtop in enumerate( xtops):
    curr_results[f'bpr_100_{y}'] = copy.deepcopy(np.mean([thing[3] for thing in xtop]))

In [70]:
curr_results

{'bpr_100_0': 0.28976780537671637, 'bpr_100_1': 0.3114375293078327}

In [73]:
pd.DataFrame(recalc_results)

,0,1,2,3,4,5,6,7
0,semi,town,2000,-auto,normal,200,0.690849,0.719448
1,semi,town,2000,-auto,normal,200,0.718195,0.734243
2,semi,town,2000,-auto-svi,normal,200,0.668744,0.688174
3,semi,town,2000,-auto-svi,normal,200,0.674319,0.688174
4,semi,town,2000,all,normal,200,0.875090,0.876031
...,...,...,...,...,...,...,...,...
64,annual,town,2000,all,normal,400,0.844269,0.842457
65,annual,town,2000,all,normal,400,0.844269,0.842457
66,annual,town,2000,-auto,poisson,400,0.823560,0.823215
67,annual,town,2000,-auto,poisson,400,0.882279,0.893014


In [11]:
recalc_results = []
test_years = 2
geography_col='geoid'
timestep_col='timestep'
outcome_col='deaths'

town_map = pd.read_csv(os.path.join(data_dir,'town_tract_map.csv'), dtype=str)
group_map = gpd.read_file(os.path.join(data_dir, 'tract_group_map'), dtype=str)


# test y always comes from quarterly tract
y_timesteps_per_year = 4
file_name = f'clean_quarter_tract'
data_path = os.path.join(data_dir, file_name)

x_idx_cols = [geography_col, 'lat','lon', timestep_col,
      'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
      'svi_pctile',
      'neighbor_t', 'self_t-1']
y_idx_cols = [geography_col, timestep_col, outcome_col]
features_only = ['lat','lon', timestep_col,
                 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
                 'svi_pctile',
                 'neighbor_t', 'self_t-1']

data_gdf = gpd.read_file(data_path)

last_train_year = 2018


test_y = data_gdf[(data_gdf['year'] > last_train_year) &
                  (data_gdf['year'] <= last_train_year+test_years)][y_idx_cols]
starting_y_timestep = int(test_y[timestep_col].min())

sorted_y_timesteps = test_y[timestep_col].unique()
sorted_y_timesteps.sort()

for loc in locs:
    for time in times:
        x_timesteps_per_year = {'quarter':4, 'semi':2,'annual':1}[time]
        
        file_name = f'clean_{time}_{loc}'
        data_path = os.path.join(data_dir, file_name)

        data_gdf = gpd.read_file(data_path)

        test_x = data_gdf[(data_gdf['year'] > last_train_year) &
                          (data_gdf['year'] <= last_train_year+test_years)][x_idx_cols]
        
        starting_x_timestep = int(test_x[timestep_col].min())
        
        test_timesteps_per_year = max(y_timesteps_per_year, x_timesteps_per_year)
        test_timesteps = test_timesteps_per_year*test_years
        
        x_repeats = int(test_timesteps_per_year/x_timesteps_per_year)
        y_repeats = int(test_timesteps_per_year/y_timesteps_per_year)
        
        sorted_x_timesteps = test_x[timestep_col].unique()
        sorted_x_timesteps.sort()
        
        x_timesteps = [timestep  for timestep in sorted_x_timesteps for _ in range(x_repeats)]
        
        y_timesteps = [timestep  for timestep in sorted_y_timesteps for _ in range(y_repeats)]

        for start_year in start_years:
            print('HNY')
            
            for num_inducing in inducing_points:
                for model in models:
                    for cov in covs:
                    
                        best_elbo = -np.inf
                        bpr_2019 = np.NaN
                        bpr_2020 = np.NaN
                        
                        for lr in learning_rates:
                            #print(lr)
                            this_run = run_template.format(time=time,loc=loc,
                                                           model=model,start_year=start_year,
                                                           cov=cov,
                                                           num_inducing=num_inducing,lr=lr)
                            try:
                                with open(os.path.join(log_dir,this_run,'model.mod'),'rb') as f:
                                        predictor = pickle.load(f)
                                with open(os.path.join(log_dir,this_run,'stats.csv'),'rb') as f:
                                    stats = pd.read_csv(f)
                                    elbo = stats.iloc[-1,:][['elbo']].values[0]
                                    
                            except(FileNotFoundError):
                                #print(f'Broke {this_run}')
                                data = [time, loc, start_year, cov, model, num_inducing, np.NaN, np.NaN]
                                recalc_results.append(data)
                                continue


                            xtops = []
                            for year in range(test_years):
                                xtop_year = []
                                max_timesteps = max(x_timesteps_per_year, y_timesteps_per_year)
                                for x_time,y_time in zip(x_timesteps[year*max_timesteps:(year+1)*max_timesteps],
                                                         y_timesteps[year*max_timesteps:(year+1)*max_timesteps]):
                                    test_x_time = test_x[test_x[timestep_col] == x_time]
                                    test_y_time = test_y[test_y[timestep_col] == y_time]
                                    _, _, _, fmean, fvar, gmean, gvar, _, _ = predictor.build_predict(test_x_time.loc[:, features_only].values)
                                    g_cond = tf.math.softplus(fmean * normcdf(gmean)).numpy()
                                    pred_df = pd.Series(g_cond.squeeze(), index=test_x_time[geography_col])
                                    
                                    if loc == 'town':
                                        merged_to_map = town_map.merge(pred_df.rename(outcome_col), right_index=True,left_on='parent_town', how='right')
                                        averaged_over_duplicates = merged_to_map.groupby('child_tracts').mean()[outcome_col]
                                        y_index = test_y[geography_col].unique()
                                        pred_df = pd.Series(index=y_index,dtype='float64')
                                        pred_df.update(averaged_over_duplicates)
                                        # not all tracts get mapped from towns
                                        pred_df=pred_df.fillna(0)
                                    elif loc == 'group':
                                        merged_to_map = group_map.merge(pred_df.rename(outcome_col), right_index=True,left_on='grouping', how='right')
                                        averaged_over_duplicates = merged_to_map.groupby('geoid').mean()[outcome_col]
                                        y_index = test_y[geography_col].unique()
                                        pred_df = pd.Series(index=y_index,dtype='float64')
                                        pred_df.update(averaged_over_duplicates)
                                        
                                    xtop_year.append(fixed_top_X(test_y_time.set_index(geography_col)[outcome_col], pred_df, 100))
                                xtops.append(xtop_year)
                            curr_results = {} 
                            for y, xtop in enumerate( xtops):
                                curr_results[f'bpr_100_{y}'] = copy.deepcopy(np.mean([thing[3] for thing in xtop]))
                                
                            bpr_2019 = copy.copy(curr_results[f'bpr_100_0'])
                            bpr_2020 = copy.copy(curr_results[f'bpr_100_1'])
                            #print(bpr_2019)
                                
                            if elbo > best_elbo:
                                best_bpr_2019 = copy.copy(bpr_2019)
                                best_bpr_2020 = copy.copy(bpr_2020)
                                best_elbo = elbo
                                

                        data = [time, loc, start_year, cov, model, num_inducing, best_bpr_2019, best_bpr_2020]
                        recalc_results.append(data)

        pd.DataFrame(recalc_results).to_csv(f'~/recalc_conv_all_q_results_sheet_{loc}_{time}.csv')
pd.DataFrame(recalc_results).to_csv('~/recalc_conv_all_q_results_sheet.csv')

HNY
HNY
HNY
HNY
HNY
HNY
HNY
HNY
HNY
HNY
HNY
HNY
HNY
HNY
HNY
HNY
HNY
HNY


In [5]:
recalc_results

[['annual',
  'tract',
  2010,
  '-auto',
  'normal',
  400,
  0.06432748538011698,
  0.048913043478260865],
 ['annual',
  'tract',
  2010,
  '-auto',
  'poisson',
  400,
  0.23391812865497075,
  0.2989130434782608],
 ['annual',
  'tract',
  2000,
  '-auto',
  'normal',
  400,
  0.4152046783625731,
  0.22826086956521732],
 ['annual',
  'tract',
  2000,
  '-auto',
  'poisson',
  400,
  0.2456140350877192,
  0.19565217391304346],
 ['semi',
  'tract',
  2010,
  '-auto',
  'normal',
  400,
  0.2783724848845277,
  0.2193775589783584],
 ['semi',
  'tract',
  2010,
  '-auto',
  'poisson',
  400,
  0.24680344930121906,
  0.16529781633846752],
 ['semi',
  'tract',
  2000,
  '-auto',
  'normal',
  400,
  0.3049856279115869,
  0.30131360889062186],
 ['semi',
  'tract',
  2000,
  '-auto',
  'poisson',
  400,
  0.32262860541183463,
  0.2827671085981672]]

In [59]:
new_zult

25001010100    42.085380
25001010206     2.357825
25001010208     2.357825
25001010304    42.516030
25001010306    17.022105
                 ...    
25027761100    42.931377
25027761200    41.283749
25027761300    41.381646
25027761401    41.406066
25027761402    41.406066
Length: 1620, dtype: float64

In [17]:
pred_df.rename('deaths').merge(town_map, left_index=True,right_on='parent_town')

AttributeError: 'Series' object has no attribute 'merge'

/tmp/ipykernel_254191/3016699809.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mapped_preds = pd.Series(index=y_index)


In [43]:
mapped_preds[[val for val in mapped_preds.index if val not in averaged_over_duplicates.index]]

25001012102    0.0
25001012200    0.0
25001014700    0.0
25001990000    0.0
25005640100    0.0
25005653301    0.0
25005990000    0.0
25009204400    0.0
25009223100    0.0
25009268400    0.0
25019950307    0.0
25019990000    0.0
25021417400    0.0
25021417501    0.0
25021417602    0.0
25021417801    0.0
25021422800    0.0
25021423102    0.0
25023500101    0.0
25023500103    0.0
25023505103    0.0
25023505104    0.0
25023505201    0.0
25023506205    0.0
25023530200    0.0
25023530300    0.0
25023530700    0.0
25023530802    0.0
25023545400    0.0
25023990003    0.0
25025060604    0.0
25025980101    0.0
25025981202    0.0
25025990101    0.0
dtype: float64

In [12]:

town_map = pd.read_csv(os.path.join(data_dir,'town_tract_map.csv'),dtype=str)
group_map = gpd.read_file(os.path.join(data_dir, 'tract_group_map'),dtype=str)

In [51]:
x_timesteps

[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0]

In [10]:
xtops

[[(111.0, 0.3508771929824561, 111.0, 0.35087719298245623),
  (132.0, 0.2542372881355932, 132.0, 0.2542372881355933),
  (143.0, 0.23529411764705882, 143.0, 0.23529411764705885),
  (141.0, 0.22950819672131148, 141.0, 0.22950819672131137)]]